In [1]:
import pandas as pd

In [8]:
allmodis = pd.read_csv(r'..\Data\Raw\archive_M6.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'daynight', 'type'], 1)
allviirs = pd.read_csv(r'..\Data\Raw\archive_V1.csv').drop(['acq_time', 'satellite', 'instrument', 'version', 'type'], 1)

In [9]:
allmodis.head()

,latitude,longitude,brightness,scan,track,acq_date,confidence,bright_t31,frp
0,-16.4439,143.5076,319.9,4.5,2.0,2000-11-01,59,300.6,62.8
1,-16.4447,143.5152,321.0,4.5,2.0,2000-11-01,62,300.5,71.6
2,-18.4459,144.8904,324.6,3.2,1.7,2000-11-01,74,303.1,77.4
3,-18.4401,144.8603,317.7,3.3,1.7,2000-11-01,61,303.2,38.5
4,-18.6804,145.5470,317.3,2.9,1.6,2000-11-01,54,299.4,32.8
